In [1]:
import sys
sys.path.append("ALL_IN_ONE/Pythia8")
sys.path.append("ALL_IN_ONE/LSD")
sys.path.append("ALL_IN_ONE/FIT_FUNC")

In [2]:
import os
from tqdm import tqdm
import ALL_IN_ONE.Pythia8.loop as loop
import ALL_IN_ONE.Pythia8.one_key_run as okr 
import ALL_IN_ONE.Pythia8.run_save as rs
import ALL_IN_ONE.Pythia8.analyse_data as ad
import ALL_IN_ONE.Pythia8.detection as detection
import ALL_IN_ONE.Pythia8.Mulity_process as mp
from datetime import datetime
import ALL_IN_ONE.Pythia8.combine as combine
# from detection import detect_percise_and_combine, detect_r_and_combine
# from analyse_data import see_ctau_2, print_max_min_log10_sin_theta_2

### This Func is made for FUTURE USE aimmed to speed up our simulations(Whose speed is Not Satisfying right now(2024.11.17)). 

### But it is not usable now

In [3]:
# from joblib import Parallel, delayed

# def worker(seed):
#     LLP_data_path = loop_mass_ctau_given_by_csv_multi(csv_file, br, seed, out_put_path, main41_path)
#     print('The Generation of LLPs is Completed')
#     completed_data_dir = detect_folder_files_no_calcu(LLP_data_path)[1]
#     print('The LLPs are Judged whether they are Detected or not')
#     final_files = combine_files_precise(completed_data_dir)
#     print('The Final Step is Over, See the .csv files for LLPs Completed Data')
#     return LLP_data_path, completed_data_dir, final_files

# def one_key_run_mass_ctau_given_by_csv_multi(csv_file, br, seed_array, out_put_path, main41_path):
#     results = Parallel(n_jobs=5)(delayed(worker)(seed) for seed in seed_array)
#     return results


## Attention:  

## The Units are CM and GEV

### The 'one_key_run_XXX' functions are all you need to use.



### Setting Parameters

In [4]:
c_mm = (3*10**5) # tau0's unit is mm/c, to convert the ultimate decay postion,
                            # we need to do sth to the ctau_typed_in 

br_lowerlim = -20 #ctau and br are in logspace
br_upperlim = -20
br_amount = 1
ctau_lowerlim = 1
ctau_upperlim = 30
ctau_amount = 50
mass = 1
seed_amount = 5
seed_array = (357993, 1234) #Pythia8 simulation seed

mass_lower_lim = 0.1 # Mass in in linspace
mass_upper_lim = 4
mass_amount = 20
br = 1

#### Same as before:  

##### The function 
```python
    one_key_run_mass_ctau_simple_ver() #Function to run an pythia8 program
```

takes 

```
br's lower / upper lim (10^x), br's amount, 

ctau's lower / upper lim (10^x), ctau's amount, 

mass, 

seed's amount, 

the Output folder's path, 

main41.cc's dir path
```
as input.

The Output is LOTS OF ```.csv``` Files with LLPs' Properties 

```
    Momentum in GeV, 
    
    lifetime(ctau or tau) in mm/C, 

    decay_postion in mm, mass in GeV 

    and so on
```

The path's format is listed below, you can change them directly to your real path to run the function

In [3]:
main131_path = '/home/srp/Documents/Pythia8/pythia8315/examples/'
# For Pythia8.31, main41 is now main131 but the path remains the same
out_path = '/home/srp/Documents/LLP_DATA/'
today = str(datetime.today().date())+'_2HDM-A'
today = '2025-06-07_2HDM-A'
print(today)


2025-06-07_2HDM-A


#### New Function info:

The function
```python
    def one_key_run_mass_ctau_br_given_by_csv_main131(csv_file, br, seed_amount, out_path, main41_path)
```

takes a ```.csv file``` as input instead of the range of ```mass```, ```tau```, ```Br_XX```and etc. 

The function will read each line of ```.csv file``` to get `LLP's DATA Range`, ALONG WITH ```ALL THE Branching Ratio of OUR LLP``` and put this set of data into pythia8. 

Pythia8 will do the simulation, then output .csv file to the ``` out_path``` folder.

In [23]:
test_csv = '/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Program/PRA/Github/position_read_analyse_1.3/ALL_IN_ONE/LSD/tau_theta_1e-05.csv'
okr.one_key_run_by_csv_cross_section_main131_simple(test_csv, 1, 1, out_path, main131_path, today, 0)

Running Simulation...


  0%|          | 0/9 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [8]:
csv_file = '/home/srp/Documents/Light_scalar_decay-main/Combined_Code/Combined_Code/2HDM-I_A_Precise2.csv'
mp.split_csv_into_different_parts(csv_file, 4)

CSV file has been split into 4 parts.


In [ ]:
test_for_mass_1 = okr.one_key_run_2HDM_cross_section_main131_lower_eff_all_detectors(csv_file, 1, 1, out_path, main131_path, today, 0)

In [4]:
print(today)

2025-06-07_2HDM-A


In [4]:
from concurrent.futures import ProcessPoolExecutor
import os

# 假设 okr.one_key_run_by_csv_cross_section_main131 是你要并行运行的函数
from concurrent.futures import ProcessPoolExecutor

def run_task(args):
    _csv_file, _param1, _param2, _out_path, _main131_path, _date, _sleep_time = args
    return okr.one_key_run_2HDMA_cross_section_main131_lower_eff_all_detectors_B(_csv_file, _param1, _param2, _out_path, _main131_path, _date, _sleep_time)

if __name__ == "__main__":
    # 自定义 CPU 核心数量
    num_cores = 4  # 你可以将此值改为你想使用的核心数量

    # 定义任务参数
    # out_path = "output_path"  # 替换为实际的输出路径
    # main131_path = "main131_path"  # 替换为实际的路径

    tasks = [
        (f"{i+1}.csv", 1, 1, out_path, main131_path, today, 3)  # 生成任务参数
        for i in range(num_cores)  # 根据自定义核心数生成任务
    ]

    # 打印生成的任务
    for task in tasks:
        print(task)

    # 使用多核并行
    with ProcessPoolExecutor(max_workers=num_cores) as executor:
        results = list(executor.map(run_task, tasks))

    # 打印结果
    print("All tasks completed:", results)


run_task((csv_file, 1, 1, out_path, main131_path, today, 0))

('1.csv', 1, 1, '/home/srp/Documents/LLP_DATA/', '/home/srp/Documents/Pythia8/pythia8315/examples/', '2025-06-07_2HDM-A', 3)
('2.csv', 1, 1, '/home/srp/Documents/LLP_DATA/', '/home/srp/Documents/Pythia8/pythia8315/examples/', '2025-06-07_2HDM-A', 3)
('3.csv', 1, 1, '/home/srp/Documents/LLP_DATA/', '/home/srp/Documents/Pythia8/pythia8315/examples/', '2025-06-07_2HDM-A', 3)
('4.csv', 1, 1, '/home/srp/Documents/LLP_DATA/', '/home/srp/Documents/Pythia8/pythia8315/examples/', '2025-06-07_2HDM-A', 3)
Running Simulation...Running Simulation...Running Simulation...Running Simulation...





100%|██████████| 156/156 [27:47:41<00:00, 641.42s/it]  


The Generation of LLPs is Completed


  1%|          | 9/1116 [01:03<2:04:36,  6.75s/it]

Error with file: /home/srp/Documents/LLP_DATA/2025-06-07_2HDM-A/LLP_data/mass_5.62e+00_ctau_7.74e+03_br_1.00e+00_seed_666783.csv
Error message: No columns to parse from file
Error with file: /home/srp/Documents/LLP_DATA/2025-06-07_2HDM-A/LLP_data/mass_5.62e+00_ctau_1.23e-03_br_1.00e+00_seed_666783.csv
Error message: No columns to parse from file


...

 55%|█████▌    | 583/1058 [09:06<07:15,  1.09it/s]

The Final Step is Over, See the .csv files for LLPs Completed Data
The Final Step is Over, See the .csv files for LLPs Completed Data


100%|██████████| 1058/1058 [14:45<00:00,  1.19it/s]


The Final Step is Over, See the .csv files for LLPs Completed Data
All tasks completed: [('/home/srp/Documents/LLP_DATA/2025-06-07_2HDM-A/LLP_data', '/home/srp/Documents/LLP_DATA/2025-06-07_2HDM-A/Completed_llp_data_precise_cross_section', ('/home/srp/Documents/LLP_DATA/2025-06-07_2HDM-A/2025-06-10_all_combined_precise_file.csv', '/home/srp/Documents/LLP_DATA/2025-06-07_2HDM-A/2025-06-10_detected_combined_precise_file.csv')), ('/home/srp/Documents/LLP_DATA/2025-06-07_2HDM-A/LLP_data', '/home/srp/Documents/LLP_DATA/2025-06-07_2HDM-A/Completed_llp_data_precise_cross_section', ('/home/srp/Documents/LLP_DATA/2025-06-07_2HDM-A/2025-06-10_all_combined_precise_file.csv', '/home/srp/Documents/LLP_DATA/2025-06-07_2HDM-A/2025-06-10_detected_combined_precise_file.csv')), ('/home/srp/Documents/LLP_DATA/2025-06-07_2HDM-A/LLP_data', '/home/srp/Documents/LLP_DATA/2025-06-07_2HDM-A/Completed_llp_data_precise_cross_section', ('/home/srp/Documents/LLP_DATA/2025-06-07_2HDM-A/2025-06-10_all_combined_preci

NameError: name 'csv_file' is not defined

#### To read the data

Now that you have done the simulation via pythia8, the output .csv files should be in your `output` folder. 

Below the folder, the file is sorted and filed by `date of the simulation`, so there will be a new folder like `2025-03-07`. In that `date folder`, there will be two folders, named`LLP_data`(store the orginal data), and `Completed_llp_data_precise` (store the detective data). 

AND THERE IS A `.csv` FILE, which is named like `2025-03-07_detected_combined_precise_file.csv`, it contains ONLY the LLPs can be DETECTED by CODEX-b.

In [ ]:
print(ad.print_max_min_log10_sin_theta_2('/Users/shiyuzhe/Documents/University/LLP/Second_Term/pythia8/BtoKa/Code/position_read_analyse_1.3/LLP_data_for_test/2025-2-16/2025-2-16_detected_combined_precise_file.csv'))

(-10.200000256677525, -12.400000474218762)


### The Detect Function

Functions::::
```python
def detect_r_and_combine(LLP_data_path) 

def detect_precise_and_combine(LLP_data_path)
```

Are functions to combine '''.csv files''' and judge whether LLP can be detected by CODEX-b or not

They will creat a ```.csv file``` at the dir of your LLP, which contains ALL DETECTED LLPS.

The function::::
```python 
def detect_r_and_combine(LLP_data_path)
```
 is merely a test function. You can play with it to get more data, but DO NOT use it for real analyzation

In [14]:
LLP_date_path = '/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/test_CODEX_Original/2025-06-01_2HDM-A/LLP_data/B_2HDM'
# detected_r = '/Users/shiyuzhe/Documents/University/LLP/Second_Term/pythia8/BtoKa/Code/position_read_analyse_1.3/LLP_data_for_test/2024-11-18_19/'
okr.calcu_cross_section_and_combine_files_CODEX_MATHUSLA(LLP_date_path)

  1%|          | 5/735 [00:13<35:32,  2.92s/it]

Error with file: /media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/test_CODEX_Original/2025-06-01_2HDM-A/LLP_data/B_2HDM/mass_1.00e+01_ctau_1.59e+01_br_1.00e+00_seed_306505.csv
Error message: No columns to parse from file


  1%|          | 7/735 [00:16<25:51,  2.13s/it]

Error with file: /media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/test_CODEX_Original/2025-06-01_2HDM-A/LLP_data/B_2HDM/mass_7.94e+00_ctau_2.00e+02_br_1.00e+00_seed_306505.csv
Error message: No columns to parse from file


  1%|          | 9/735 [00:19<21:37,  1.79s/it]

...

100%|██████████| 735/735 [29:35<00:00,  2.42s/it]


The LLPs are Judged whether they are Detected or not, and calculated the cross section


100%|██████████| 595/595 [06:50<00:00,  1.45it/s]


The Final Step is Over, See the .csv files for LLPs Completed Data


('/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/test_CODEX_Original/2025-06-01_2HDM-A/LLP_data/Completed_llp_data_precise_cross_section',
 ('/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/test_CODEX_Original/2025-06-01_2HDM-A/LLP_data/2025-06-03_all_combined_precise_file.csv',
  '/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/test_CODEX_Original/2025-06-01_2HDM-A/LLP_data/2025-06-03_detected_combined_precise_file.csv'))

In [ ]:
combinefile = '/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/'
combine.merge_csv_files(combinefile, combinefile + '/combined.csv')

All CSV files have been merged into /media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/ALL_GOOD_DATA_2/combined.csv
